In [56]:
import pandas as pd
import numpy as np
import scipy as sc

import sklearn as sk
from sklearn.model_selection import KFold
 

# Чтение базы фильмов

In [2]:
%%time
movie_titles = pd.read_csv(r'D:\Bugaychenko\Netflix\movie_titles.csv', 
                           encoding = 'ISO-8859-1', 
                           header = None, 
                           names = ['Id', 'Year', 'Name']).set_index('Id')

movie_titles = movie_titles.drop(movie_titles[movie_titles.Year.isnull()].index)
movie_titles.Year = movie_titles.Year.astype(int)

print(movie_titles)

       Year                                               Name
Id                                                            
1      2003                                    Dinosaur Planet
2      2004                         Isle of Man TT 2004 Review
3      1997                                          Character
4      1994                       Paula Abdul's Get Up & Dance
5      2004                           The Rise and Fall of ECW
6      1997                                               Sick
7      1992                                              8 Man
8      2004                         What the #$*! Do We Know!?
9      1991                           Class of Nuke 'Em High 2
10     2001                                            Fighter
11     1999                     Full Frame: Documentary Shorts
12     1947                               My Favorite Brunette
13     2003  Lord of the Rings: The Return of the King: Ext...
14     1982                                 Nature: Ant

# Чтение юзер даты

In [2]:
%%time
k =1
for i in range(1,5):
    df_raw = pd.read_csv(r'D:\Bugaychenko\Netflix\combined_data_{}.txt'.format(i), header=None, 
                                                                            names=['User', 'Rating', 'Date'], 
                                                                            usecols=[0, 1, 2])
    index_films = df_raw[(df_raw.Date.isnull())& (df_raw.Rating.isnull())].index # промежутки с оценками на фильмы
    df_raw = df_raw.drop('Date', 1)
    #df_raw = df_raw.drop('Rating', 1)
    # создание столбца №фильма
    a = np.empty(len(df_raw), dtype=int)
    for current, next in zip(index_films, index_films[1:]):
        a[current:next] = k
        k+=1
        a[next:] = k
    df_raw["Movie"] = a
    df = df_raw.drop(index_films) #удаляем пустые строки
    df.Rating = df.Rating.astype(int)
    #df = df[df.Movie.isin(movie_titles.index.unique().values)] # проверка присутствия фильмов
    
    if(i == 1):
        df_merge = df
    else:
        df_merge = df_merge.append(df, ignore_index=True)
        
    print(len(df_merge))
    #y.append(df.Rating.values) # таргет
    #shpe_df.append(df.shape[0])
    #unic_user.append(df.User.unique())
    #np.save('target2', y) 
    
    del df_raw
    del index_films
    del a
    del df
    print(i)
    #del y
   
    #### убрать при пересоздании спарсы
    #del df
    #del movie_titles

24053764
1
51031355
2
73632984
3
100480507
4
Wall time: 3min 16s


In [3]:
%%time
import sys
print(sys.getsizeof(df_merge))
#np.save('target_all',df_merge.Rating.values)
df_merge = df_merge.drop(df_merge.Rating, axis=1)
print(sys.getsizeof(df_merge))

Wall time: 7min 40s


# Создание спарс матрицы



In [3]:
%%time
print(df_merge)
col1 = df_merge.Movie.unique().shape[0]
#col1 = 17767
print(col1)
col2 = df_merge.User.unique().shape[0]
#col2 = 480189
print(col2)
X = df_merge[:24053764].shape[0]
print(X)
#del df_merge


              User  Rating  Movie
0          1488844       3      1
1           822109       5      1
2           885013       4      1
3            30878       4      1
4           823519       3      1
5           893988       3      1
6           124105       4      1
7          1248029       3      1
8          1842128       4      1
9          2238063       3      1
10         1503895       4      1
11         2207774       5      1
12         2590061       3      1
13            2442       3      1
14          543865       4      1
15         1209119       4      1
16          804919       4      1
17         1086807       3      1
18         1711859       4      1
19          372233       5      1
20         1080361       3      1
21         1245640       3      1
22          558634       4      1
23         2165002       4      1
24         1181550       3      1
25         1227322       4      1
26          427928       4      1
27          814701       5      1
28          80

In [17]:
X = df_merge[73632984:].shape[0]

In [18]:
%%time
sparse_dataset = sc.sparse.lil_matrix((X, col1 + col2))

Wall time: 16.7 s


In [5]:
%%time
customer = {j:i for i, j in enumerate(df_merge.User.unique())}
movie = {j:i for i, j in enumerate(df_merge.Movie.unique())}



Wall time: 52.5 s


In [19]:
%%time
for i, j in enumerate(df_merge[73632984:].itertuples()):
    sparse_dataset[i, customer[j.User]] = 1
    sparse_dataset[i, col1 + movie[j.Movie]] = 1

Wall time: 11min 31s


In [20]:
%%time
sparse_dataset = sc.sparse.csr_matrix(sparse_dataset)
sc.sparse.save_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super4.npz', sparse_dataset)

Wall time: 10min 11s


# Загрузка спарс сета


In [2]:
sparse_dataset1 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super1.npz')
sparse_dataset2 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super2.npz')
sparse_dataset3 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super3.npz')
sparse_dataset4 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super4.npz')

sparse_dataset = sc.sparse.vstack((sparse_dataset1, sparse_dataset2, sparse_dataset3, sparse_dataset4))

y = np.load('target_all.npy').reshape(-1, 1)

del sparse_dataset1
del sparse_dataset2
del sparse_dataset3
del sparse_dataset4


#sparse_dataset = joblib.load('sparse_df_2.bin')
#

In [4]:
sparse_dataset

<100480507x497956 sparse matrix of type '<class 'numpy.float64'>'
	with 200960424 stored elements in Compressed Sparse Row format>

In [5]:
len(y)

100480507

### FM

In [22]:
def RMSE_score(y_pred, y_real):
    return np.sqrt(np.sum(np.power(y_pred - y_real, 2)) / y_pred.shape[0])

def lr_decrease(lr,lr_dec,epochs):
    lr = lr * np.exp(-lr_dec*epochs)
    return lr

def Nonlinear(X):
    global Z
    global X_squared
    X_squared = X.power(2)
    Z = X @ V
    #1/2 (X*V)^2 - X^2*V^2
    return 1/2 * np.sum(
        np.power(Z, 2) - X_squared @ np.power(V, 2),axis=1, keepdims=True)
        
def predict(X):
    return w0 + X @ w1 + Nonlinear(X)
    
def fit(X, Y, mini_batches=5 , batch_size = 100, lr=0.001, lr_dec=0.001, k=5):
    global w0 
    global w1 
    global V
    
    
    w0 = 0
    w1 = np.random.normal(size=(X.shape[1], 1), scale=.1)
    V = np.random.normal(size=(X.shape[1], k), scale=.1)
    n = X.shape[0]
    lr_new = lr
    
    for epoch in range(0, mini_batches):
        batch_indices = np.random.choice(n, batch_size)
        x = X[batch_indices, :]
        y = Y[batch_indices]
        y_hat = predict(x)
        dy = 2 * (y_hat - y) 
        w0 -= np.mean(dy) * lr_new 
        w1 -= x.multiply(dy).mean(axis=0).T * lr_new
        for i in range(k):
            left = x.multiply(Z[:, i].reshape(-1, 1)) #X*XV
            right = X_squared.multiply(V[:, i]) #X^2*V
            V[:, i] -= (np.asarray((left - right).multiply(dy).mean(axis=0).T) * lr_new).flatten() 
        lr_new = lr_decrease(lr,lr_dec,epoch)
        if(epoch %10000 == 0):
            print(epoch)
            print(lr_new)

In [23]:
%%time
FM2 = fit(sparse_dataset, y, mini_batches=sparse_dataset.shape[0]//1000, batch_size=1000, lr=0.3, lr_dec=0.00001, k=2)

0
0.3
10000
0.27145122541078787
20000
0.24561922592339452
30000
0.22224546620451535
40000
0.20109601381069178
50000
0.18195919791379003
60000
0.16464349082820792
70000
0.14897559113742284
80000
0.13479868923516647
90000
0.12197089792217973
100000
0.1103638323514327
Wall time: 1h 46min 48s


In [28]:
sk.metrics.mean_squared_error(y[:50000000], predict(sparse_dataset[:50000000]))

0.9839960865827178

In [29]:
sk.metrics.mean_squared_error(y[50000000:], predict(sparse_dataset[50000000:]))

0.9811352453519191

In [30]:
np.sqrt((0.9839960865827178+0.9811352453519191)/2)

0.9912445036252753

In [50]:
sparse_dataset1 = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset_super1.npz')

In [55]:
np.sqrt(sk.metrics.mean_squared_error(y[:sparse_dataset1.shape[0]], predict(sparse_dataset1)))

0.9937534158189794